In [2]:
from google.colab import drive
drive.mount('/content/drive')

import pickle

with open('/content/drive/MyDrive/df_model.pkl', 'rb') as f:
    df_model = pickle.load(f)

df_model.head()


Mounted at /content/drive


,title,description,job_skills,job_industries,pay_period,max_salary,med_salary,min_salary,normalized_salary,job_benefit_categories,...,city,fips,name,description_company,address,follower_count,currency,company_size_label,posting_days,remote_allowed_dummy
0,Marketing Coordinator,Job descriptionA leading real estate firm in N...,Marketing; Sales,Real Estate,HOURLY,20.0,76030.085761,17.0,38480.0,Pension; health_insurance,...,Jersey City,34021.0,Corcoran Sawyer Smith,With years of experience helping local buyers ...,242 Tenth Street,2351.0,USD,50–160 employees,30.0,0.0
1,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",Health Care Provider,Unknown,HOURLY,50.0,40.000000,30.0,83200.0,Pension; health_insurance,...,New York,8069.0,Unknown,Unknown,Unknown,37313.0,USD,9000–43000 employees,30.0,0.0
2,Assitant Restaurant Manager,The National Exemplar is accepting application...,Management; Manufacturing,Restaurants,YEARLY,65000.0,47076.583656,45000.0,55000.0,Pension,...,Mariemont,39061.0,The National Exemplar,"In April of 1983, The National Exemplar began ...",6880 Wooster Pike,40.0,USD,10–70 employees,30.0,0.0
3,Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,Other,Law Practice,YEARLY,175000.0,66496.078048,140000.0,157500.0,Pension,...,Lake Success,36059.0,"Abrams Fensterman, LLP","Abrams Fensterman, LLP is a full-service law f...",3 Dakota Drive,2427.0,USD,50–160 employees,30.0,0.0
4,Service Technician,Looking for HVAC service tech with experience ...,Information Technology,Facilities Services,YEARLY,80000.0,70000.000000,60000.0,70000.0,Pension; health_insurance,...,New York,19057.0,Unknown,Unknown,Unknown,37313.0,USD,9000–43000 employees,30.0,0.0


In [5]:
cat_cols = [
    "job_skills",
    "job_industries",
    "pay_period",
    "job_benefit_categories",
    "formatted_work_type",
    "application_type",
    "formatted_experience_level",
    "company_name",
    "industry_group_15",
    "state_region_5",
    "country_grouped",
    "city",
    "currency",
    "company_size_label"
]

# Count unique categories for each column
for col in cat_cols:
    n_unique = df_model[col].nunique(dropna=True)
    print(f"{col}: {n_unique} unique categories")


job_skills: 1412 unique categories
job_industries: 2712 unique categories
pay_period: 5 unique categories
job_benefit_categories: 15 unique categories
formatted_work_type: 7 unique categories
application_type: 4 unique categories
formatted_experience_level: 6 unique categories
company_name: 10172 unique categories
industry_group_15: 16 unique categories
state_region_5: 6 unique categories
country_grouped: 4 unique categories
city: 2216 unique categories
currency: 6 unique categories
company_size_label: 7 unique categories


In [7]:
import pandas as pd

cat_summary = pd.DataFrame({
    "column": cat_cols,
    "n_unique": [df_model[c].nunique(dropna=True) for c in cat_cols]
})

cat_summary.sort_values("n_unique", ascending=False)


,column,n_unique
7,company_name,10172
1,job_industries,2712
11,city,2216
0,job_skills,1412
8,industry_group_15,16
3,job_benefit_categories,15
4,formatted_work_type,7
13,company_size_label,7
9,state_region_5,6
6,formatted_experience_level,6


In [12]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Columns to encode
ohe_cols = [
    "industry_group_15",
    "job_benefit_categories",
    "formatted_work_type",
    "company_size_label",
    "state_region_5",
    "formatted_experience_level",
    "currency",
    "pay_period",
    "application_type",
    "country_grouped",
]

# Create a working copy
df_ohe = df_model.copy()

# Identify columns that will NOT be encoded
non_ohe_cols = [c for c in df_ohe.columns if c not in ohe_cols]

# Build the encoder
enc = OneHotEncoder(
    handle_unknown="ignore",   # ignore unseen categories at inference
    drop=None,                 # keep all categories
    sparse_output=False,       # return dense array
    dtype=np.uint8             # memory efficient
)

# Fit + transform only categorical subset
X_cat = df_ohe[ohe_cols].astype("string")
X_enc = enc.fit_transform(X_cat)

# Build encoded DataFrame
enc_cols = enc.get_feature_names_out(ohe_cols)
df_enc = pd.DataFrame(X_enc, columns=enc_cols, index=df_ohe.index)

# Combine with the rest of the dataset
df_ohe_final = pd.concat([df_ohe[non_ohe_cols], df_enc], axis=1)

# Sanity check
print("Original shape:", df_model.shape)
print("After OHE shape:", df_ohe_final.shape)
print("Encoded columns added:", len(enc_cols))


Original shape: (35121, 37)
After OHE shape: (35121, 103)
Encoded columns added: 76


In [13]:
df_ohe_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35121 entries, 0 to 35120
Columns: 103 entries, title to country_grouped_Unknown
dtypes: category(4), float64(17), object(6), uint8(76)
memory usage: 9.1+ MB


In [14]:
# import pickle

# # Save the encoded dataframe
# df_ohe_final.to_pickle("df_ohe_final.pkl")

# # Save the encoder object
# with open("onehot_encoder.pkl", "wb") as f:
#     pickle.dump(enc, f)

# print("Saved locally in the notebook directory.")


Saved locally in the notebook directory.


In [15]:
# from google.colab import drive
# drive.mount('/content/drive')

# # Save df_ohe_final to Drive
# df_ohe_final.to_pickle("/content/drive/MyDrive/df_ohe_final.pkl")

# # Save the encoder to Drive
# with open("/content/drive/MyDrive/onehot_encoder.pkl", "wb") as f:
#     pickle.dump(enc, f)

# print("Saved to Google Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved to Google Drive.


In [16]:
# from google.colab import files

# # Download df_ohe_final
# files.download("df_ohe_final.pkl")

# # Download the encoder
# files.download("onehot_encoder.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>